In [42]:
import sys
sys.path.append('../src')
from logger import log, debug
from cleaner import clean_corpus_basic_human
from reader import read_files
from lstm_utils import get_tokenizer, get_best_tokens_dummy

from sklearn.utils import shuffle

from keras.layers import Dense, LSTM
from keras.models import Model, Sequential, load_model

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats




In [43]:
# -------------- Hiperparametros -----------------

vector_size = 8
each_q = 128
batch_size = 2000
epoch = 256
min_lenght, max_length = 4, 8
train_to_test = 0.8

In [44]:
human_corpus = pd.DataFrame()
count_failed = []

for i in range(10):
    # Leemos los archivos
    corpus_raw = pd.read_csv("./samples/human/pos_neg_" + str(i + 1) + ".csv")
    corpus_raw["failed"] = corpus_raw["rate"] != corpus_raw["human_rate"]

    # guardamos cuantos fallaron
    count_failed.append(len(corpus_raw[corpus_raw["failed"] == True]))
    
    # Limpiamos el dataframe
    human_corpus = human_corpus.append(clean_corpus_basic_human(corpus_raw))

# Filtramos por longitud
debug("[El corpus tiene " + str(len(human_corpus)) + " rows]")

#Shuffleamos el corpus
human_corpus = shuffle(human_corpus)
human_corpus.reset_index(inplace=True, drop=True)


pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 200)

human_corpus.head()


[Usando cleaner basico para samples clasificados por humanos]
[Limpiando el corpus clasificado por humanos...]
[Usando 8 threads ...]
[Usando cleaner basico para samples clasificados por humanos]
[Limpiando el corpus clasificado por humanos...]
[Usando 8 threads ...]
[Usando cleaner basico para samples clasificados por humanos]
[Limpiando el corpus clasificado por humanos...]
[Usando 8 threads ...]
[Usando cleaner basico para samples clasificados por humanos]
[Limpiando el corpus clasificado por humanos...]
[Usando 8 threads ...]
[Usando cleaner basico para samples clasificados por humanos]
[Limpiando el corpus clasificado por humanos...]
[Usando 8 threads ...]
[Usando cleaner basico para samples clasificados por humanos]
[Limpiando el corpus clasificado por humanos...]
[Usando 8 threads ...]
[Usando cleaner basico para samples clasificados por humanos]
[Limpiando el corpus clasificado por humanos...]
[Usando 8 threads ...]
[Usando cleaner basico para samples clasificados por humanos]


,content,sentiment,raw,rate,human_rate
0,PROLIJ DESPACH LIND BICI BUEN RELACION PRECI CALID,"[0, 1]","Muy prolijos con el despacho Muy linda bici, buena relación entre precio y calidad",positive,positive
1,MEJOR VAS TERMIC VAS ESTA TERRIBL CUMPL TOTAL ESPECT MEJOR,"[0, 1]",Uno de los mejores vasos termicos!!! El vaso está terrible. Cumple totalmente con las espectativas. Es uno de los mejores.,positive,positive
2,BUEN COLOR TAMAÑ ESTAN BIEN,"[0, 1]",muy bueno El color y tamaño están muy bien,positive,positive
3,MAL FALT POTENCI VERD NO RECOM DEJ CARG VERD NAD POTENCI ATORNILL FUE DESPACH COM CORRESPOND TIEMP FORM AHI NO HAY QUEJ,"[0, 1]","Malo Faltó de potencia la verdad no la recomiendo, la dejé cargando y la verdad nada 0 potencia para atornillar, el producto fue despachado como corresponde en tiempo y forma ahí no hay quejas!!!",negative,negative
4,MALISSIM MALISSIM ENCASTR PIL MAL NO FUNCION DESILUCION NO RECOM NAD,"[0, 1]",malissimo el producto. El producto es malissimo. desde el encastre de las pilas es malo y no funciona . una desilucion el producto. NO lo recomiendo para nada.,negative,negative


In [55]:
# Calculamos la posibilidad de que un rate sea failed
debug(count_failed)

std = np.std(count_failed)
mean = np.mean(count_failed)
distr = scipy.stats.norm(mean, std)

debug("[El standard deviation de las muestras failed es: " + str(std) + "]")
debug("[El mean de las muestras failed es: " + str(mean) + "]")
debug("[La probabilidad de que el error esté entre un 3% y un 7% es " + \
      str(distr.cdf(7) - distr.cdf(3)) + \
      "]")
      

[8, 4, 1, 3, 7, 10, 3, 4, 8, 6]
[El standard deviation de las muestras failed es: 2.690724809414742]
[El mean de las muestras failed es: 5.4]
[La probabilidad de que el error esté entre un 3% y un 7% es 0.5377472973487742]


In [46]:
# Seleccionamos los mejores tokens
best_tokens = pd.read_pickle("best_tokens.bin")


max_features = best_tokens.size

tokenizer = get_tokenizer(best_tokens, vector_size)

# Vectorizamos
debug("[Vectorizando corpus... ]")
human_corpus['vector'] = tokenizer(human_corpus['content'])
#del neutral_corpus['content']


[Vectorizando corpus... ]


In [47]:
# Extraemos los x
x = np.stack(human_corpus["vector"].values)
# Cargamos el model
model = load_model("./model.h5")


In [48]:
def to_rate(row):
    percentage = row[0]
    if(percentage <= 0.5):
        return "negative"
    return "positive"

def to_percentage(row):
    percentage = row[0]
    return '%.3f'%(row[0] * 100) + "%"
    

In [49]:
# Hacemos prediccion contra el model
y = model.predict(x, batch_size=batch_size)

#5.7962010e-05, 9.9994206e-01
human_corpus["prediction"] = y.tolist()
human_corpus["machine_rate"] = human_corpus["prediction"].apply(to_rate)


In [51]:
# Vemos acc del la predicción
rates_diff = human_corpus[human_corpus["human_rate"] != human_corpus["machine_rate"]].copy()
rates_ok = human_corpus[human_corpus["human_rate"] == human_corpus["machine_rate"]].copy()

err = len(rates_diff.index) / len(human_corpus.index)
debug("[El accuaracy en el corpus corregido es de: " + str(1 - err) + "]")


[El accuaracy en el corpus corregido es de: 0.969]
